In [147]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Embedding, CuDNNLSTM, Bidirectional
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.initializers import glorot_uniform

# Loading the Dataset

In [2]:
df = pd.read_csv('data/data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(10)


,Handle,Party,Tweet
0,RepDarrenSoto,Democrat,Gov. @ricardorossello's comments degrading wom...
1,RepDarrenSoto,Democrat,Looks like Trump will end his discriminatory c...
2,RepDarrenSoto,Democrat,For several years we sought to replace our sta...
3,RepDarrenSoto,Democrat,Proud to announce that @fema awarded @PolkCoun...
4,RepDarrenSoto,Democrat,STURDY Bill passed @EnergyCommerce Cmte today ...
5,RepDarrenSoto,Democrat,Enough is enough! Billionaire super predator J...
6,RepDarrenSoto,Democrat,We continue our efforts to provide American Ci...
7,RepDarrenSoto,Democrat,We're committed to defending quality &amp; aff...
8,RepDarrenSoto,Democrat,Robocalls aren’t just annoying. Many are outri...
9,RepDarrenSoto,Democrat,This is why we continue to fight @jediabetical...


# Pre-processing

In [3]:
df['Party'] = pd.Categorical(df.Party)
df['Party'] = pd.get_dummies(df['Party'], drop_first=True)
# df[df['Party'] == 0]
df.head()


,Handle,Party,Tweet
0,RepDarrenSoto,0,Gov. @ricardorossello's comments degrading wom...
1,RepDarrenSoto,0,Looks like Trump will end his discriminatory c...
2,RepDarrenSoto,0,For several years we sought to replace our sta...
3,RepDarrenSoto,0,Proud to announce that @fema awarded @PolkCoun...
4,RepDarrenSoto,0,STURDY Bill passed @EnergyCommerce Cmte today ...


<b>0</b> - Democrat <br>
<b>1</b> - Republican

In [4]:
X = df['Tweet']
Y = df['Party']
print(X.shape)
print(Y.shape)

(90000,)
(90000,)


In [5]:
print(X.iloc[0])

Gov. @ricardorossello's comments degrading women, including my dear friend @MMViverito, are unacceptable. I condemn these demeaning words. Now more than ever, Puerto Rico is in need of strong leadership. I urge the Governor to use appropriate language &amp; always respect women.


### Removing special characters

In [6]:
X = X.str.replace('&amp;', "and")
X = X.str.replace('\xa0', " ")
X = X.str.replace('\u2003', " ")


In [7]:
print(X.iloc[122])

We’re indeed in a constitutional crisis and must defend our democracy! AG Barr should be held in contempt for refusal to testify and submit docs. As co-equal branch, Congress has a constitutional obligation to oversight, combat corruption and get to the truth!
https://t.co/vjZ2P7PSWl


### Removing links

In [8]:
def remove_link(sent):
    sent = str(sent)
    return sent[:sent.find("https://")]

In [9]:
X = X.apply(remove_link)

In [10]:
print(X.iloc[122])

We’re indeed in a constitutional crisis and must defend our democracy! AG Barr should be held in contempt for refusal to testify and submit docs. As co-equal branch, Congress has a constitutional obligation to oversight, combat corruption and get to the truth!



### Removing punctuations and converting sentences into lowercase

In [19]:
import re
from string import punctuation
punctuation = punctuation + "—\n\t"
regex = re.compile('[%s]' % re.escape(punctuation))

def remove_punctuations(sentence):
    return regex.sub('', sentence)
    

In [20]:
X = X.apply(remove_punctuations)
X = X.apply(str.lower)

In [21]:
print(X.iloc[122])

we’re indeed in a constitutional crisis and must defend our democracy ag barr should be held in contempt for refusal to testify and submit docs as coequal branch congress has a constitutional obligation to oversight combat corruption and get to the truth


In [22]:
X.loc[1024]

'what we saw today was horrifying that’s why we need to continue to shine a light on what’s happening at our southern border  we have to make sure that when congress allocates funding there are oversight provisions msnbc allinwithchris '

In [77]:
ind = list(X[X==""].index)
x = X.drop(ind)
y = Y.drop(ind)
y

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
89970    0
89971    0
89972    0
89973    0
89974    0
89975    0
89976    0
89977    0
89978    0
89979    0
89980    0
89981    0
89982    0
89983    0
89984    0
89985    0
89986    0
89987    0
89988    0
89989    0
89990    0
89991    0
89992    0
89993    0
89994    0
89995    0
89996    0
89997    0
89998    0
89999    0
Name: Party, Length: 89541, dtype: uint8

# Modelling
<b><hr>

## Splitting into training and testing sets


In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=420)

## Using Tokenizer and creating a vocabulary

In [79]:
t = Tokenizer(oov_token="UNK")
a = list(x)

In [80]:
print("Len of a: {}".format(len(a)))

Len of a: 89541


### Creating a vocabulary

In [81]:
t.fit_on_texts(a)

In [82]:
vocab_size = len(t.word_index) + 1
print("Vocabulary size: {}".format(vocab_size))

Vocabulary size: 89911


In [83]:
max_sent_len = len(max(X, key=len).split()) + 1
print("Maximum sentence length: {}".format(max_sent_len))

Maximum sentence length: 74


### Encoding the sentences

In [119]:
def convert_sentences(sentence, word_to_index):
    encoded = np.zeros((1, max_sent_len))
    sentence_words = sentence.lower().split()
    j = 0
    for w in sentence_words:
        encoded[0, j] = word_to_index[w]
        j += 1
    
    return np.float32(encoded)

In [128]:
encoded_X_train = t.texts_to_sequences(X_train)

In [137]:
padded_X_train = pad_sequences(encoded_X_train, maxlen=max_sent_len, padding='post')
padded_X_train

array([[  99,   93, 1529, ...,    0,    0,    0],
       [   2,  839,   10, ...,    0,    0,    0],
       [1126, 1192, 5953, ...,    0,    0,    0],
       ...,
       [   2,   41,  271, ...,    0,    0,    0],
       [  26,   10, 2651, ...,    0,    0,    0],
       [ 128,   13,  590, ...,    0,    0,    0]])

## Embedding Layer

In [44]:
emb_dim = 50

In [138]:
embedding_layer = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_sent_len, trainable=False)
trained_embedding_layer = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_sent_len, trainable=True)

In [139]:
# encoded_X_train1 = tf.convert_to_tensor(encoded_X_train, np.float32)
layer_1 = embedding_layer(padded_X_train)

In [140]:
trained_layer_1 = trained_embedding_layer(padded_X_train)

In [144]:
n_classes = 2
learning_rate = 0.05
epochs = 50
num_hidden_units = 128
timesteps = 28

In [148]:
model = Sequential([
    embedding_layer,
    Bidirectional(CuDNNLSTM(128, return_sequences=True)),
    Dropout(0.5),
    CuDNNLSTM(128),
    Dropout(0.5),
    Dense(2),
    Activation('softmax')
])

W0718 00:53:54.259601 20456 deprecation.py:506] From C:\Users\tshan\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 00:53:54.263591 20456 deprecation.py:506] From C:\Users\tshan\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 00:53:54.264588 20456 deprecation.py:506] From C:\Users\tshan\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_

In [149]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 74, 50)            4495550   
_________________________________________________________________
bidirectional (Bidirectional (None, 74, 256)           184320    
_________________________________________________________________
dropout (Dropout)            (None, 74, 256)           0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               197632    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
_________________________________________________________________
activation (Activation)      (None, 2)                 0

In [152]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [154]:
model.fit(padded_X_train, y_train, epochs=50, batch_size = 400, shuffle=True)

Epoch 1/50
71632/71632 [==============================] - 29s 398us/sample - loss: 0.6933 - acc: 0.4984
Epoch 2/50
71632/71632 [==============================] - 28s 398us/sample - loss: 0.6932 - acc: 0.5016
Epoch 3/50
71632/71632 [==============================] - 35s 489us/sample - loss: 0.6932 - acc: 0.5009
Epoch 4/50
71632/71632 [==============================] - 35s 483us/sample - loss: 0.6931 - acc: 0.5029
Epoch 5/50
71632/71632 [==============================] - 31s 435us/sample - loss: 0.6932 - acc: 0.4982
Epoch 6/50
71632/71632 [==============================] - 32s 446us/sample - loss: 0.6932 - acc: 0.4979
Epoch 7/50
71632/71632 [==============================] - 39s 545us/sample - loss: 0.6932 - acc: 0.5007
Epoch 8/50
71632/71632 [==============================] - 33s 466us/sample - loss: 0.6932 - acc: 0.4970
Epoch 9/50
71632/71632 [==============================] - 34s 475us/sample - loss: 0.6932 - acc: 0.5004
Epoch 10/50
71632/71632 [==============================] - 35s 4

In [162]:
model2 = Sequential([
    embedding_layer,
    Bidirectional(CuDNNLSTM(128, return_sequences=False)),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

In [163]:
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [164]:
model2.fit(padded_X_train, y_train, epochs=100, batch_size = 400, shuffle=True)

Epoch 1/100
71632/71632 [==============================] - 5s 77us/sample - loss: 0.6890 - acc: 0.5358
Epoch 2/100
71632/71632 [==============================] - 5s 70us/sample - loss: 0.6865 - acc: 0.5494
Epoch 3/100
71632/71632 [==============================] - 5s 72us/sample - loss: 0.6833 - acc: 0.5588
Epoch 4/100
71632/71632 [==============================] - 5s 73us/sample - loss: 0.6809 - acc: 0.56540s - loss: 0.6809 - acc: 0
Epoch 5/100
71632/71632 [==============================] - 6s 82us/sample - loss: 0.6802 - acc: 0.5651
Epoch 6/100
71632/71632 [==============================] - 6s 82us/sample - loss: 0.6782 - acc: 0.5712
Epoch 7/100
71632/71632 [==============================] - 6s 83us/sample - loss: 0.6780 - acc: 0.5690
Epoch 8/100
71632/71632 [==============================] - 6s 82us/sample - loss: 0.6763 - acc: 0.5723
Epoch 9/100
71632/71632 [==============================] - 6s 83us/sample - loss: 0.6756 - acc: 0.5742
Epoch 10/100
71632/71632 [=====================

71632/71632 [==============================] - 8s 114us/sample - loss: 0.5360 - acc: 0.7164
Epoch 79/100
71632/71632 [==============================] - 7s 95us/sample - loss: 0.5306 - acc: 0.7204
Epoch 80/100
71632/71632 [==============================] - 6s 90us/sample - loss: 0.5288 - acc: 0.7210
Epoch 81/100
71632/71632 [==============================] - 6s 91us/sample - loss: 0.5238 - acc: 0.7253
Epoch 82/100
71632/71632 [==============================] - 9s 128us/sample - loss: 0.5182 - acc: 0.7281
Epoch 83/100
71632/71632 [==============================] - 8s 117us/sample - loss: 0.5156 - acc: 0.7311
Epoch 84/100
71632/71632 [==============================] - 7s 100us/sample - loss: 0.5187 - acc: 0.7283
Epoch 85/100
71632/71632 [==============================] - 7s 99us/sample - loss: 0.5139 - acc: 0.7326
Epoch 86/100
71632/71632 [==============================] - 7s 100us/sample - loss: 0.5074 - acc: 0.7360
Epoch 87/100
71632/71632 [==============================] - 7s 100us/sam

In [ ]:
model3 = Sequential([
    embedding_layer,
    Bidirectional(CuDNNLSTM(256, return_sequences=False)),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(padded_X_train, y_train, epochs=150, batch_size = 400, shuffle=True)

Epoch 1/150
71632/71632 [==============================] - 11s 160us/sample - loss: 0.6892 - acc: 0.5384
Epoch 2/150
71632/71632 [==============================] - 11s 155us/sample - loss: 0.6859 - acc: 0.5510 - loss: 0.6860 - acc:
Epoch 3/150
71632/71632 [==============================] - 11s 155us/sample - loss: 0.6825 - acc: 0.5606
Epoch 4/150
71632/71632 [==============================] - 11s 156us/sample - loss: 0.6812 - acc: 0.5657
Epoch 5/150
71632/71632 [==============================] - 11s 157us/sample - loss: 0.6798 - acc: 0.5664 - loss: 0.6799 - acc:
Epoch 6/150
71632/71632 [==============================] - 11s 157us/sample - loss: 0.6783 - acc: 0.5686 - loss: 0.6782 - acc
Epoch 7/150
71632/71632 [==============================] - 11s 158us/sample - loss: 0.6776 - acc: 0.5690
Epoch 8/150
71632/71632 [==============================] - 11s 158us/sample - loss: 0.6807 - acc: 0.5608
Epoch 9/150
71632/71632 [==============================] - 11s 159us/sample - loss: 0.6776 - ac

def weight_variable(shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W',
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b',
                           dtype=tf.float32,
                           initializer=initial)

def BiRNN(x, weights, biases, timesteps, num_hidden):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get BiRNN cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                                 dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights) + biases